In [23]:
import pandas as pd
import os
import folium
from folium import plugins
from pyproj import Transformer

In [12]:
PATH = os.getcwd()

In [14]:
df = pd.read_csv(f"{PATH}/data/ruido.csv", sep=";")
df.head()


,NMT,anio,mes,dia,tipo,LAEQ,DIRECCION,X (ETRS89),Y (ETRS89)
0,1,2021,1,1,D,62.6,Paseo de Recoletos Frente al nº 23 del Paseo d...,441303,4474895
1,1,2021,1,1,E,64.4,Paseo de Recoletos Frente al nº 23 del Paseo d...,441303,4474895
2,1,2021,1,1,N,62.6,Paseo de Recoletos Frente al nº 23 del Paseo d...,441303,4474895
3,1,2021,1,1,T,63.3,Paseo de Recoletos Frente al nº 23 del Paseo d...,441303,4474895
4,2,2021,1,1,D,65.8,Glorieta de Carlos V Plaza del Emperador Carlo...,441328,4473396


In [24]:
# Creamos el transformador de ETRS89 UTM zona 30N (EPSG:25830) a WGS84 (EPSG:4326)
transformer = Transformer.from_crs("EPSG:25830", "EPSG:4326")

# Añadimos las columnas de latitud y longitud al dataframe
df['lat'], df['lon'] = transformer.transform(df['X (ETRS89)'], df['Y (ETRS89)'])

df.head()


,NMT,anio,mes,dia,tipo,LAEQ,DIRECCION,X (ETRS89),Y (ETRS89),lat,lon
0,1,2021,1,1,D,62.6,Paseo de Recoletos Frente al nº 23 del Paseo d...,441303,4474895,40.422616,-3.691925
1,1,2021,1,1,E,64.4,Paseo de Recoletos Frente al nº 23 del Paseo d...,441303,4474895,40.422616,-3.691925
2,1,2021,1,1,N,62.6,Paseo de Recoletos Frente al nº 23 del Paseo d...,441303,4474895,40.422616,-3.691925
3,1,2021,1,1,T,63.3,Paseo de Recoletos Frente al nº 23 del Paseo d...,441303,4474895,40.422616,-3.691925
4,2,2021,1,1,D,65.8,Glorieta de Carlos V Plaza del Emperador Carlo...,441328,4473396,40.409114,-3.691492


In [36]:
# Creamos un mapa centrado en Madrid
mapa_madrid = folium.Map(
    location=[40.4168, -3.7038],  # Coordenadas del centro de Madrid
    zoom_start=12
)

marker_cluster = plugins.MarkerCluster().add_to(mapa_madrid)

# Visualización usando las coordenadas convertidas
for idx, row in df.drop_duplicates(subset=['NMT']).iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=f"Estación: {row['DIRECCION']}<br>Ruido promedio: {df_calor[df_calor['NMT']==row['NMT']]['LAEQ'].values[0]:.1f} dB",
    ).add_to(marker_cluster)

display(mapa_madrid)

In [37]:
# 1. Mapa de calor del ruido promedio
df_promedio = df.groupby('NMT')['LAEQ'].mean().reset_index()
df_ubicaciones = df.drop_duplicates(subset=['NMT'])[['NMT', 'lat', 'lon']]
df_calor = pd.merge(df_promedio, df_ubicaciones, on='NMT')

# Crear el mapa de calor
mapa_calor = folium.Map(location=[40.4168, -3.7038], zoom_start=12)

# Añadir el mapa de calor al mapa
plugins.HeatMap(df_calor[['lat', 'lon', 'LAEQ']].values.tolist()).add_to(mapa_calor)

# Mostrar el mapa
display(mapa_calor)


In [41]:
# 3. Mapa con círculos proporcionales al nivel de ruido normalizado
mapa_circulos = folium.Map(
    location=[40.4168, -3.7038],
    zoom_start=12
)

# Calculamos el rango de valores para normalizar
min_db = df_calor['LAEQ'].min()
max_db = df_calor['LAEQ'].max()

# Definimos el rango de tamaños que queremos para los círculos
min_radius = 5
max_radius = 30

for idx, row in df_calor.iterrows():
    # Normalizamos el radio entre min_radius y max_radius
    radius = min_radius + (row['LAEQ'] - min_db) * (max_radius - min_radius) / (max_db - min_db)
    
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=radius,
        popup=f"Estación: {df[df['NMT']==row['NMT']]['DIRECCION'].iloc[0]}<br>Ruido: {row['LAEQ']:.1f} dB",
        color='blue',
        fill=True,
        fill_color='blue',
        tooltip=f"{row['LAEQ']:.1f} dB"  # Añadido tooltip para ver el valor al pasar el mouse
    ).add_to(mapa_circulos)

display(mapa_circulos)

In [44]:
# Creamos el mapa base
mapa_combinado = folium.Map(
    location=[40.4168, -3.7038],
    zoom_start=12
)

# 1. Capa de clusters de marcadores
marker_cluster = plugins.MarkerCluster(name='Estaciones (clusters)').add_to(mapa_combinado)

for idx, row in df.drop_duplicates(subset=['NMT']).iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=f"Estación: {row['DIRECCION']}<br>Ruido promedio: {df_calor[df_calor['NMT']==row['NMT']]['LAEQ'].values[0]:.1f} dB",
    ).add_to(marker_cluster)

# 2. Capa de mapa de calor
heat_data = [[row['lat'], row['lon'], row['LAEQ']] for idx, row in df_calor.iterrows()]
heat_group = folium.FeatureGroup(name='Mapa de calor')
plugins.HeatMap(heat_data).add_to(heat_group)
heat_group.add_to(mapa_combinado)

# 3. Capa de círculos proporcionales
circle_group = folium.FeatureGroup(name='Círculos de ruido')

min_db = df_calor['LAEQ'].min()
max_db = df_calor['LAEQ'].max()
min_radius = 5
max_radius = 30

for idx, row in df_calor.iterrows():
    radius = min_radius + (row['LAEQ'] - min_db) * (max_radius - min_radius) / (max_db - min_db)
    
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=radius,
        popup=f"Estación: {df[df['NMT']==row['NMT']]['DIRECCION'].iloc[0]}<br>Ruido: {row['LAEQ']:.1f} dB",
        color='blue',
        fill=True,
        fill_color='blue',
        tooltip=f"{row['LAEQ']:.1f} dB"
    ).add_to(circle_group)

circle_group.add_to(mapa_combinado)

# Añadimos el control de capas
folium.LayerControl().add_to(mapa_combinado)

# Mostramos el mapa
display(mapa_combinado)

In [48]:
# Creamos el mapa base
mapa_combinado = folium.Map(
    location=[40.4168, -3.7038],
    zoom_start=12
)

# Definimos colores para cada tipo de medición
colores_tipo = {
    'D': 'blue',      # Diurno
    'E': 'green',     # Vespertino
    'N': 'red',       # Nocturno
    'T': 'purple'     # Total
}

# Definimos etiquetas descriptivas para cada tipo
etiquetas_tipo = {
    'D': 'Diurno',
    'E': 'Vespertino',
    'N': 'Nocturno',
    'T': 'Total'
}

# Iteramos por tipo
for tipo in sorted(df['tipo'].unique()):
    # Filtramos datos para este tipo
    df_filtrado = df[df['tipo'] == tipo]
    df_promedio = df_filtrado.groupby('NMT')['LAEQ'].mean().reset_index()
    df_ubicaciones = df_filtrado.drop_duplicates(subset=['NMT'])[['NMT', 'lat', 'lon', 'DIRECCION']]
    df_calor = pd.merge(df_promedio, df_ubicaciones, on='NMT')
    
    # 2. Capa de mapa de calor
    heat_data = [[row['lat'], row['lon'], row['LAEQ']] for idx, row in df_calor.iterrows()]
    heat_group = folium.FeatureGroup(name=f'Calor {etiquetas_tipo[tipo]}')
    plugins.HeatMap(heat_data).add_to(heat_group)
    heat_group.add_to(mapa_combinado)
    

# Añadimos el control de capas
folium.LayerControl(collapsed=False).add_to(mapa_combinado)

# Mostramos el mapa
display(mapa_combinado)